In [162]:
import numpy as np
import pandas as pd

### Exploratory Analysis

First, for the exploratory analysis, I want to internalize the data, understand the column names and their types. I need to know what I am working with. Then, I will look at the column variables and brainstorm some insights that the data is able to show me. Let's start by reading the csv file and taking a look at the variable labels and variable types



In [201]:
data = pd.read_csv('/Users/HuanLiang/Downloads/projects.csv')
list(data)

['projectid',
 'teacher_acctid',
 'schoolid',
 'school_ncesid',
 'school_latitude',
 'school_longitude',
 'school_city',
 'school_state',
 'school_zip',
 'school_metro',
 'school_district',
 'school_county',
 'school_charter',
 'school_magnet',
 'school_year_round',
 'school_nlns',
 'school_kipp',
 'school_charter_ready_promise',
 'teacher_prefix',
 'teacher_teach_for_america',
 'teacher_ny_teaching_fellow',
 'primary_focus_subject',
 'primary_focus_area',
 'secondary_focus_subject',
 'secondary_focus_area',
 'resource_type',
 'poverty_level',
 'grade_level',
 'fulfillment_labor_materials',
 'total_price_excluding_optional_support',
 'total_price_including_optional_support',
 'students_reached',
 'eligible_double_your_impact_match',
 'eligible_almost_home_match',
 'date_posted']

In [202]:
data.dtypes

projectid                                  object
teacher_acctid                             object
schoolid                                   object
school_ncesid                             float64
school_latitude                           float64
school_longitude                          float64
school_city                                object
school_state                               object
school_zip                                float64
school_metro                               object
school_district                            object
school_county                              object
school_charter                             object
school_magnet                              object
school_year_round                          object
school_nlns                                object
school_kipp                                object
school_charter_ready_promise               object
teacher_prefix                             object
teacher_teach_for_america                  object


First, I find it odd that the school_ncesid is a float but the other ids are objects. I suppose that it is just the way that different organizations identify each project. After reading what each category of data means on the Kaggle website and understanding the category labels, I can start my exploratory analysis. First, how many projects are there? What is the range of dates that the projects are recorded? 

In [203]:
data.shape

(664098, 35)

Straight away, I see missing values. I want to remove those points so I do not introduce unwanted noise in my analysis. 

In [166]:
data = data.dropna()
data.shape

(365074, 35)

Almost half the entires are dropped. In the future, I will want to take time to investigate whether all rows with a missing value needs to be dropped, since I am losing signal as well when I drop that data. For example, we don't know who fills the data in, so there might be a relationship between the date or location of the project and the amount of data that is filled in for the project. For example, I noticed that the fulfillment cost was not filled during the earlier years, and the numbers of students reached was also zero. 

Now, since I am interested in the geographical locations of the projects, what states and counties have the greatest number of projects?

In [204]:
data['school_state'].value_counts()[:5]

CA    126242
NY     73182
NC     43478
IL     40167
TX     39661
Name: school_state, dtype: int64

In [205]:
data['school_county'].value_counts()[:5]

Los Angeles             50223
Cook                    31458
Kings (Brooklyn)        19551
Bronx                   18121
New York (Manhattan)    13954
Name: school_county, dtype: int64

It makes sense that most of the projects are located in large urban areas, such as Chicago, New York and Los Angeles. However, I can't be sure if the program targets city schools or if city schools are just more in need. I see that there are categorical variables that describe the school. What percentage is true for the categories? I consider 'Mr.' as true for teacher_prefix.

In [206]:
l1 = data.loc[:, 'school_charter':'teacher_ny_teaching_fellow']
for col in l1:
    if col == 'teacher_prefix':
        print(col, data.loc[data[col] == 'Mr.'].shape[0] / data.shape[0]) 
    else:
        print(col, data.loc[data[col] == 't'].shape[0] / data.shape[0])

school_charter 0.08672966941626085
school_magnet 0.09376628148255227
school_year_round 0.04973814105749453
school_nlns 0.014969176236037452
school_kipp 0.006616493348873208
school_charter_ready_promise 0.004899879234691266
teacher_prefix 0.12794497197702748
teacher_teach_for_america 0.0568425142072405
teacher_ny_teaching_fellow 0.01722336161229216


I see that most cases are false for the categories. For example, only eight percent of the project schools are charter schools. Surprisingly, only thirteen percent of the teachers at project schools are men. Next, I want to see which subjects and focus areas come up the most often in the projects. 

In [207]:
l2 = data.loc[:,'primary_focus_subject': 'fulfillment_labor_materials']
for col in l2: 
    print(col)
    print(data[col].value_counts() )
    print()

primary_focus_subject
Literacy                 201378
Literature & Writing      82189
Mathematics               80929
Special Needs             41959
Applied Sciences          31615
Visual Arts               30738
Environmental Science     27178
Health & Life Science     24617
Music                     20590
History & Geography       16861
Early Development         14140
Other                     13459
Social Sciences           10024
ESL                        9790
Character Education        9571
Performing Arts            8565
Gym & Fitness              7050
Health & Wellness          6713
College & Career Prep      6521
Foreign Languages          5401
Sports                     3557
Extracurricular            3094
Civics & Government        2440
Economics                  1806
Community Service          1491
Parent Involvement         1258
Nutrition                  1125
Name: primary_focus_subject, dtype: int64

primary_focus_area
Literacy & Language    298758
Math & Science        

I see that literacy is the main focus area for a great amount of the projects. This may be related to that most projects help younger kids, since it is important for younger children to learn to read and write. Lastly, I want to check out the average price of the optional support.

In [171]:
difference = data['total_price_including_optional_support'] - data['total_price_excluding_optional_support']
difference.mean()

100.0339210680606

### A Data Story 

Something I think that is interesting is the relationship between poverty level and geographic location. Specifically, I want to tell a story based on how the highest level of poverty is most common in certain areas. Then, I want to take a look at Illinois and Cook county to see which zip codes have the highest rate of 'highest poverty' projects.

People usually tend to believe that school districts with the highest poverty level are urban school districts. Taking a look at our data, we are able to see if it is true of not. 

In [208]:
for i in data['poverty_level'].unique():
    print(i)
    print(data[data['poverty_level'] == i]['school_metro'].value_counts(normalize=True))

highest poverty
urban       0.721913
suburban    0.192854
rural       0.085234
Name: school_metro, dtype: float64
high poverty
urban       0.470299
suburban    0.308128
rural       0.221573
Name: school_metro, dtype: float64
moderate poverty
suburban    0.433711
urban       0.348684
rural       0.217605
Name: school_metro, dtype: float64
low poverty
suburban    0.512800
urban       0.363396
rural       0.123805
Name: school_metro, dtype: float64


In the projects, it seems to be true, since over 70 percent of the highest poverty schools are urban, compared with a 35 percent of low poverty schools that are urban. Let's dive deeper into the highest poverty and urban category. What resource type is given to 'highest poverty' level? 

In [199]:
data[data['poverty_level'] == 'highest poverty']['resource_type'].value_counts(normalize=True)[:5]

Supplies      0.342469
Technology    0.328004
Books         0.214614
Other         0.104914
Trips         0.008554
Name: resource_type, dtype: float64

It seems that supplies and technology are the top two resources for highest poverty schools. What about geographical setting?

In [209]:
data[data['poverty_level'] == 'highest poverty']['school_state'].value_counts(normalize=True)[:5]

CA    0.170954
NY    0.156993
IL    0.081598
TX    0.058015
NC    0.053071
Name: school_state, dtype: float64

Compared with the states with the most cases, which came in at CA, NY, TX, IL, and NC in that order, it appears that Illinois and Texas are flipped. As a result, I suspect that a greater number of projects in Illinois is categorized as 'highest poverty' than Texas. As a student at the University of Chicago, I understand that there is great wealth inequality in the city. Let's see if I am correct.

In [180]:
data[data['school_state'] == 'IL']['poverty_level'].value_counts(normalize = True)

highest poverty     0.783750
high poverty        0.112176
moderate poverty    0.094351
low poverty         0.009723
Name: poverty_level, dtype: float64

In [181]:
data[data['school_state'] == 'TX']['poverty_level'].value_counts(normalize = True)

highest poverty     0.555425
high poverty        0.312980
moderate poverty    0.104691
low poverty         0.026904
Name: poverty_level, dtype: float64

The difference between Illinois and Texas in the highest poverty level is more than 20 percentage points! Now, let's continue the story by seeing which county in Illinois has the greatest "highest poverty" rate. My guess is Cook county.

In [185]:
data[(data['school_state'] == 'IL') & (data['poverty_level'] == 'highest poverty')]['school_county'].value_counts(normalize = True)

Cook           0.888404
Lake           0.028717
Sangamon       0.016197
Champaign      0.010511
Winnebago      0.010051
Kane           0.009247
Will           0.008213
Peoria         0.005399
St Clair       0.004480
Macon          0.003848
Rock Island    0.003733
Vermilion      0.002240
Madison        0.002068
Du Page        0.001608
Mclean         0.001091
Hancock        0.000689
Kankakee       0.000689
Pulaski        0.000574
Tazewell       0.000459
Boone          0.000459
Union          0.000287
Fulton         0.000287
Massac         0.000172
De Kalb        0.000172
Henderson      0.000172
Morgan         0.000115
Kendall        0.000057
Pike           0.000057
Name: school_county, dtype: float64

I was correct. With my previous knowledge that most 'highest poverty' cases are urban, I could have guess that Chicago's county has the highest rate. Now we know that Cook county has the most cases, which zip code is especially burdened?

In [192]:
data[(data['school_county'] == 'Cook') & ((data['poverty_level'] == 'highest poverty'))]['school_zip'].value_counts(normalize = True)

60609.0    0.061991
60623.0    0.055139
60652.0    0.051067
60647.0    0.040918
60618.0    0.040853
60632.0    0.037427
60625.0    0.035100
60612.0    0.033937
60660.0    0.032321
60636.0    0.030640
60637.0    0.029089
60617.0    0.027925
60624.0    0.026115
60629.0    0.024887
60608.0    0.023400
60634.0    0.022948
60641.0    0.021913
60621.0    0.021590
60628.0    0.021202
60626.0    0.020814
60651.0    0.020750
60616.0    0.019910
60639.0    0.019845
60622.0    0.018940
60402.0    0.016807
60640.0    0.016742
60620.0    0.015320
60619.0    0.015191
60644.0    0.015191
60610.0    0.014932
             ...   
60458.0    0.001616
60429.0    0.001616
60473.0    0.001228
60707.0    0.001164
60104.0    0.000970
60466.0    0.000970
60160.0    0.000711
60090.0    0.000582
60425.0    0.000517
60452.0    0.000517
60056.0    0.000452
60164.0    0.000388
60655.0    0.000388
60631.0    0.000323
60018.0    0.000323
60501.0    0.000323
60471.0    0.000259
60428.0    0.000194
60472.0    0.000194


There isn't one specific zip that stands out. However, I would be interested in learning more about the first four zip codes and their demographics. I would also like to learn about those schools.

### Questions for the Project Partner

Now, especially after I looked into my data story, I want to know the social economic status of the inhabitants of the top five zip codes in Cook county where schools are in the 'highest poverty' level. If I have that information, then I am able to find the relationship between poverty level and social economic status of the zip code. This goes along for all schools that are projects in the spreadsheet. While I know that the poverty level is dependent on percentage of students who get reduced lunches, it is also useful to have a concrete number, such as median income earned in the zip area. 

Another question I have for my project partner is what conditions make a school eligible to become a project? Like I stated earlier, urban schools dominate the projects, but we know that there are rural areas that need help as well. 
In addition, there is little quantitative data about the projects overall. For example, what was the outcome measure after the project was completed? Did this see an increase in the rate of graduation or college attendance? Since many of the projects took place at primary schools, did the boost for the younger children sustain across the rest of their education?